<big> constants, imports

In [1]:
#constants
filename="chatbot_model"
TLD = "/content/drive/My Drive/TensorBoard_logs/{}/".format(filename)
bash_TLD = "/content/drive/My\ Drive/TensorBoard_logs/{}/".format(filename)
data_folder = 'drive/My Drive/Data/RC/'

from google.colab import drive
drive.mount('/content/drive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#import utils
import os
from time import time
from keras.callbacks import TensorBoard
import tensorflow as tf
import keras
from keras.models import Model
from keras.models import Sequential
from keras import layers
from keras import losses
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import numpy as np
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten, Input, LSTM, Dense, GRU
from keras.layers.embeddings import Embedding
from keras.layers import Lambda
from keras.models import load_model
from keras.backend import argmax

from tensorflow.python.client import device_lib

from keras import backend as K
import pandas as pd

from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.saved_model.signature_def_utils_impl import predict_signature_def

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

#import keras.layers.LSTM


from sklearn.model_selection import train_test_split


start_token = 'starttoken'
end_token = 'endtoken'

number_of_classes = 22

vocab_size = 40000


embedding_vector_dim = 300

max_sequence_length = 20

max_decoder_seq_length = 25


chatbot_ds_file = 'combined1.txt'

# path = ''
# Glove_path = 'glove.6B.'+str(embedding_vector_dim)+'d.txt'
# mount google drive
path = '/content/drive/My Drive/Data/chatbot_general_data/'
# Glove_path = '/content/drive/My Drive/Data/Glove/glove.6B.'+str(embedding_vector_dim)+'d.txt'
word2vec_path = '/content/drive/My Drive/Data/fastText/wiki-news-300d-1M-subword.vec'

Using TensorFlow backend.


In [0]:
#don't worry about this, this is just for tensorboard data

import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.python.eager import context

class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir="/content/drive/My Drive/TensorBoard_logs/{}".format(filename), **kwargs):
        self.val_log_dir = os.path.join(log_dir, 'validation')
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

    def set_model(self, model):
        if context.executing_eagerly():
            self.val_writer = tf.contrib.summary.create_file_writer(self.val_log_dir)
        else:
            self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def _write_custom_summaries(self, step, logs=None):
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if 'val_' in k}
        if context.executing_eagerly():
            with self.val_writer.as_default(), tf.contrib.summary.always_record_summaries():
                for name, value in val_logs.items():
                    tf.contrib.summary.scalar(name, value.item(), step=step)
        else:
            for name, value in val_logs.items():
                summary = tf.Summary()
                summary_value = summary.value.add()
                summary_value.simple_value = value.item()
                summary_value.tag = name
                self.val_writer.add_summary(summary, step)
        self.val_writer.flush()

        logs = {k: v for k, v in logs.items() if not 'val_' in k}
        super(TrainValTensorBoard, self)._write_custom_summaries(step, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()


In [5]:
#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip ngrok-stable-linux-amd64.zip

def tensorBoard():
  #LOG_DIR = './content/drive/My Drive/TensorBoard_logs/{}'.format(filename)
  LOG_DIR = "./log"
  !rsync -a $bash_TLD "./log"
  
  tensorboard = TensorBoard(log_dir=LOG_DIR)
  print('LOG_DIR:',LOG_DIR)
  get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
   )
  get_ipython().system_raw('./ngrok http 6006 &')
  !curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

#use this link to enter tensorBoard
tensorBoard()

LOG_DIR: ./log
https://97727985.ngrok.io


<big> loading data

In [0]:
#input training data to chatbot           sample line: question+'\t'+answer
def input_chatbot_data(fileName):
  '''
  Args:
    fileName (str): file to load
  Return:
    init (list): list of questions
    ip_res (list): list of answers pre-padded with start_token
    op_res (list): list of answers post-padded woth end_token
  '''
  init = []
  ip_res = []
  op_res = []
  with open(fileName, encoding='iso-8859-1') as f:
    for line in f:
      lineSplit = line.split('\t')
      Q = lineSplit[0]
      if(len(Q) >= 2 and (Q[-1] == '.' or Q[-1] == '?' or Q[-1] == '!')): Q = Q[:-1] + ' ' + Q[-1]
      init.append(Q)
      
      Q = lineSplit[1]
      if(len(Q) >= 2 and (Q[-2] == '.' or Q[-2] == '?' or Q[-2] == '!')): Q = Q[:-2] + ' ' + Q[-2]
      ip_res.append(start_token + ' ' + Q)
      op_res.append(Q + ' ' + end_token)
  return init,ip_res,op_res

In [0]:
def save_df(name):
  path = data_folder + name
  df.to_csv(path_or_buf= data_folder + name, sep='&')

def load_df(name):
  return pd.read_csv(data_folder + name, sep='&',index_col=0)

In [0]:
df = load_df('20k_df')

In [0]:
def add_tokens(data):
  init = data['comment']
  ip_res = start_token + ' ' + data['reply']
  op_res = data['reply'] + ' '+ end_token
  
  return init,ip_res,op_res


<big> embeddings


In [0]:
def get_tokenizer(X):
    ##TOKENIZE ON SPACES ONLY? HOW WILL IT AFFECT t.texts_to_sequences(df)
    t = Tokenizer()
    t.fit_on_texts(X)
    return t


In [0]:
t = get_tokenizer(index_to_word)
# print(t.word_index["the"])

vocab_size = len(t.word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_vector_dim))
for word, i in t.word_index.items():
  if(i==0): print('oh no')
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector
    
    
# flag = True
# for index in range(1,total_vocab_size):
#     flag = flag and ( t.index_word[index] == index_to_word[index] )
    

   

    


In [23]:
print('Indexing word vectors.')

#embedding_matrix = np.zeros((total_vocab_size, embedding_vector_dim))

index_to_word = []
word_to_index = {}
embeddings_index = {}
f = open(word2vec_path, encoding='utf-8')
for index, line in enumerate(f):
    if index < vocab_size :
        values = line.split()
        word = values[0]
        index_to_word.append(word)
        word_to_index[word] = index
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
        #embedding_matrix[index + 1] = coefs
    else:
      break
f.close()

embeddings_index[start_token] = np.zeros(embedding_vector_dim) + 0.0015
index_to_word.append(start_token)

embeddings_index[end_token] = np.zeros(embedding_vector_dim) - 0.0015
index_to_word.append(end_token)

#print('Found %s word vectors.' % len(embedding_matrix))



Indexing word vectors.


In [24]:
##adding embbeding into tensorBoard

def embeddings_to_TLD():
  import numpy as np
  embeddings_vectors = np.stack(embedding_matrix)
  emb = tf.Variable(embeddings_vectors, name='word_embeddings')
  init_op = tf.global_variables_initializer()
  saver = tf.train.Saver()
  with tf.Session() as sess:
    sess.run(init_op)
    save_path = saver.save(sess,"/content/drive/My Drive/TensorBoard_logs/{}/model.ckpt".format(filename))
    print("Model saved in path: %s" % save_path)

  words = '\n'.join(index_to_word)
  with open(os.path.join("/content/drive/My Drive/TensorBoard_logs/{}".format(filename), 'metadata.tsv'), 'w') as f:
     f.write(words)
embeddings_to_TLD()

Model saved in path: /content/drive/My Drive/TensorBoard_logs/chatbot_model/model.ckpt


In [0]:
#device_lib.list_local_devices()
# def download_file(): 
#   !wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.en.vec
#   !mv -v /content/*.vec '/content/drive/My Drive/Data/fastText'
  
# download_file()


In [0]:
# #outputs numerically tokenized words

# def preprocessInputSentences(sentences):
#     tokenized_sentences = t.texts_to_sequences(sentences)
#     print(tokenized_sentences)
#     print(type(tokenized_sentences))
#     #list of lists to numpy array of arrays
#     tokenized_sentences = np.array([np.array(sentence) for sentence in tokenized_sentences])
#     tokenized_sentences = tokenized_sentences[tokenized_sentences != 0]
#     tokenized_sentences -= 1
#     print(tokenized_sentences)
#     print(type(tokenized_sentences))
#     tokenized_sentences =  [element for element  in array for array in tokenized_sentences.tolist()]
#     print(tokenized_sentences)
#     print(type(tokenized_sentences[0][0]))
#     return pad_sequences(tokenized_sentences, maxlen = max_sequence_length, padding='post')

<big> padding, model and training

In [25]:
# '''
# initiation = ["hi how are you", "whats your name", "how much is this watch"]
# input_response = [start_token+" i am fine thank you", start_token+" my name is fadi", start_token+" its so expensive you cant afford"]
# output_response = ["i am fine thank you "+end_token, "my name is fadi "+end_token,"its so expensive you cant afford "+end_token]
# '''

#initiation,input_response,output_response = input_chatbot_data(path + chatbot_ds_file)
initiation,input_response,output_response = add_tokens(df)


encoder_input_data = pad_sequences(t.texts_to_sequences(initiation), maxlen = max_sequence_length, padding='post')

decoder_input_data = pad_sequences(t.texts_to_sequences(input_response), maxlen = max_sequence_length, padding='post')

decoder_target_data = pad_sequences(t.texts_to_sequences(output_response), maxlen = max_sequence_length, padding='post')

#decoder_target_data = to_categorical((decoder_target_data), num_classes = total_vocab_size + 1)

# print("")
# print(to_categorical([-1,-1,3,0]))
encoder_input_data_train, encoder_input_data_validation, decoder_input_data_train, decoder_input_data_validation, decoder_target_data_train, decoder_target_data_validation = train_test_split(encoder_input_data, decoder_input_data, decoder_target_data, test_size = 0.1, random_state = 42)

print(len(encoder_input_data_train))

18000


In [0]:

latent_dim = 200

bottleneck_size = 100

In [29]:
tf.reset_default_graph()
sess = tf.Session(config = config)
K.set_session(sess)


# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,),name="encoder_inputs")
embeddingLayer = Embedding(vocab_size, embedding_vector_dim, weights=[embedding_matrix], trainable = False,name="EmbeddingLayer")
embeddingForInput = embeddingLayer(encoder_inputs)
encoder = LSTM(latent_dim, return_state=True,name="LSTM_Encoder")
encoder_outputs, state_h, state_c = encoder(embeddingForInput)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
# decoder_inputs = Input(shape=(None, total_vocab_size))
# relu_decoder_input_bottleneck_layer = Dense(bottleneck_size, activation='relu')
# relu_decoder_input_bottleneck_layer_output = relu_decoder_input_bottleneck_layer(decoder_inputs)
decoder_inputs = Input(shape=(None,),name="decoder_inputs")
embeddingForOutput = embeddingLayer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,name="LSTM_Decoder")
decoder_outputs, _, _ = decoder_lstm(embeddingForOutput,
                                     initial_state=encoder_states)
relu_decoder_output_bottleneck_layer = Dense(bottleneck_size, activation='relu')
relu_decoder_output_bottleneck_layer_output = relu_decoder_output_bottleneck_layer(decoder_outputs)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(relu_decoder_output_bottleneck_layer_output)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
#model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

writer = tf.summary.FileWriter(TLD, graph=sess.graph)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_inputs (InputLayer)     (None, None)         0                                            
__________________________________________________________________________________________________
encoder_inputs (InputLayer)     (None, None)         0                                            
__________________________________________________________________________________________________
EmbeddingLayer (Embedding)      (None, None, 300)    9576900     encoder_inputs[0][0]             
                                                                 decoder_inputs[0][0]             
__________________________________________________________________________________________________
LSTM_Encoder (LSTM)             [(None, 200), (None, 400800      EmbeddingLayer[0][0]             
__________

In [0]:
def generator(encoder_input, decoder_input, decoder_output, batch_size):

    while True:
        batch_start = 0
        total_length = len(encoder_input)
        while batch_start < total_length:
          batch_end = min(batch_start + batch_size, total_length)
          yield [encoder_input[batch_start : batch_end], decoder_input[batch_start : batch_end]], to_categorical(decoder_output[batch_start : batch_end], num_classes = vocab_size)
          batch_start += batch_size

In [0]:
batch_size = 125
epochs = 3


In [0]:
# Run training
model.compile(optimizer='adam', loss=losses.categorical_crossentropy, metrics=['categorical_accuracy'])
#model.fit([encoder_input_data_train, decoder_input_data_train], to_categorical(decoder_target_data_train, num_classes = vocab_size),
#          batch_size = batch_size, epochs=epochs)

#writer = tf.summary.FileWriter('./log', graph=sess.graph)

model.fit_generator(generator(encoder_input_data_train, decoder_input_data_train, decoder_target_data_train, batch_size),
                    steps_per_epoch = int(len(encoder_input_data_train) / batch_size),                 
                    epochs = epochs,
                    validation_data = generator(encoder_input_data_validation, decoder_input_data_validation, decoder_target_data_validation, batch_size),
                    validation_steps = int(len(encoder_input_data_validation) / batch_size), callbacks=[TrainValTensorBoard(write_graph=False)])



Epoch 1/3
144/144 [==============================] - 70s 487ms/step - loss: 5.2124 - categorical_accuracy: 0.4119 - val_loss: 4.1302 - val_categorical_accuracy: 0.4323
Epoch 2/3
144/144 [==============================] - 65s 451ms/step - loss: 3.9600 - categorical_accuracy: 0.4392 - val_loss: 3.8577 - val_categorical_accuracy: 0.4649
Epoch 3/3
144/144 [==============================] - 65s 452ms/step - loss: 3.8128 - categorical_accuracy: 0.4626 - val_loss: 3.7922 - val_categorical_accuracy: 0.4772


In [0]:
def to_savedmodel(model, export_path):
  """Convert the Keras HDF5 model into TensorFlow SavedModel."""

  builder = saved_model_builder.SavedModelBuilder(export_path)
  outputs_dict =  {'output' + str(index) :  output for index, output in enumerate(model.outputs)}
  signature = predict_signature_def(
      inputs={'encoder_input': model.inputs[0], 'input_token': model.inputs[1] }, outputs = outputs_dict)

  with K.get_session() as sess:
    builder.add_meta_graph_and_variables(
        sess=sess,
        tags=[tag_constants.SERVING],
        signature_def_map={
            signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature
        })
    builder.save()

In [0]:
# loss, accuracy = model.evaluate([encoder_input_data_test, decoder_input_data_test], to_categorical(decoder_target_data_test), batch_size = len(encoder_input_data_test))
# print('test Accuracy: %f' % (accuracy*100))

In [0]:
def myfunction(x):
  return x[0, 0]

In [0]:
argmax_layer =  Lambda(argmax)

current_state = encoder_states

input_token = Input(shape=(None,), name = "input_token")
current_input = input_token
decoder_outputs = []

for i in range(max_sequence_length):
    embedding = embeddingLayer(current_input)
    decoder_output, h, c = decoder_lstm(embedding, initial_state = current_state)
    decoder_output = relu_decoder_output_bottleneck_layer(decoder_output)
    decoder_output = decoder_dense(decoder_output)
    decoder_output = argmax_layer(decoder_output)
    current_input = decoder_output
    current_state = h, c
    decoder_outputs.append(decoder_output)


 
      

model_p = Model(inputs = [encoder_inputs, input_token] , outputs = decoder_outputs)

#this looks wrong, check tensorBoard
writer = tf.summary.FileWriter('./log', graph=sess.graph)



In [0]:
model_p.inputs

In [0]:

#model.save('full_chatbot_model_test1_dataset.h5')
#K.set_learning_phase(0)
#to_savedmodel(model_p, '')





In [0]:
#encoder_model = load_model('encoder_model.h5')
#decoder_model = load_model('decoder_model.h5')
#print(encoder_model.summary())
# print(decoder_model.summary())

In [0]:
def decode_sequence(input_seq):
  sentence = ''
  end_token_index = t.word_index[end_token]
  start_token_index = np.full((1, 1), fill_value = t.word_index[start_token])
  for element in model_p.predict([input_seq, start_token_index]):
    element = element[0, 0]
    if (element == 0 or element == end_token_index):
      break
    sentence += ' ' +  t.index_word[element]
  return sentence

In [0]:
print(decode_sequence(pad_sequences(t.texts_to_sequences(["hi"]), maxlen = max_sequence_length, padding='post')))

 i think you


In [0]:
print(pad_sequences(t.texts_to_sequences(["hi"]), maxlen = max_sequence_length, padding='post'))
print(t.word_index[start_token])

In [0]:
# #loading glove
# #note: should be dumbed into drive to not download it everytime
# def download(): 
#   !wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki-news-300d-1M-subword.vec.zip
#   !unzip wiki*.zip
#   #To move all .txt from colab to drive files, but not folders:
#   !mv -v /content/*.vec '/content/drive/My Drive/Data/fastText'

# download()

In [0]:
from math import log

BEAM_LENGTH = 1
BEAM_COUNT_EACH_TIME_STEP = 1

def beamSearch (previousTokenIndex, previousDecoderState, cumulativeSentenceProbability, beamLength):
    
    #stopping conditions, either local beam length is finished or we faced end token

    
    if previousTokenIndex == t.word_index[end_token]:
        #normalize beam probability if we face unequal beam lengths due to end_token
        averageWordProbability = 2**(log(cumulativeSentenceProbability,2)/(BEAM_LENGTH-beamLength))
        cumulativeSentenceProbability = averageWordProbability**BEAM_LENGTH
        previousDecoderState = None #to raise errors if used again
        beamLength = 0#to ensure going to the second if statement
    
    if beamLength <= 0:
        return ([], previousDecoderState, cumulativeSentenceProbability)
    
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = previousTokenIndex
    output_tokens, h, c = decoder_model.predict([target_seq] + previousDecoderState)
    sorted_indices = np.argsort(output_tokens)
    
    filteredIndices = np.zeros((BEAM_COUNT_EACH_TIME_STEP), dtype=int)
    filteredProbabilities = np.zeros((BEAM_COUNT_EACH_TIME_STEP))
    for beamLocalID in range(BEAM_COUNT_EACH_TIME_STEP):
        filteredIndices[beamLocalID] = sorted_indices[0,0,-(1+beamLocalID)]
        filteredProbabilities[beamLocalID] = output_tokens[0,-1,filteredIndices[beamLocalID]]
        
    
    #delete to save memory on recursion
    del output_tokens 
    del previousDecoderState
    
    maximumProbabilitySentenceIndices = []
    maximumProbabilityBeamLastDecoderState = None
    maximumProbabilityBeamCumulativeSentenceProbability = -66666666
    
    bestTokenIndex = -1
    
    for beamLocalID in range(BEAM_COUNT_EACH_TIME_STEP):
        current_token_index = filteredIndices[beamLocalID]
        current_token_index_prediction_probability = filteredProbabilities[beamLocalID]
        sentenceIndices,lastDecoderState,currentCumulativeSentenceProbability = beamSearch(current_token_index,[h, c],cumulativeSentenceProbability*current_token_index_prediction_probability*10,beamLength-1)
        if currentCumulativeSentenceProbability > maximumProbabilityBeamCumulativeSentenceProbability:
            bestTokenIndex = current_token_index
            maximumProbabilityBeamLastDecoderState = lastDecoderState
            maximumProbabilitySentenceIndices = sentenceIndices
            maximumProbabilityBeamCumulativeSentenceProbability = currentCumulativeSentenceProbability
            
    if bestTokenIndex != 0 and bestTokenIndex != t.word_index[end_token]:
        sentenceIndices.insert(0,bestTokenIndex)
    return (sentenceIndices,maximumProbabilityBeamLastDecoderState,maximumProbabilityBeamCumulativeSentenceProbability)
    

In [0]:
def decode_sequence_with_beamsearch(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    sentenceIndices = []
    cumulativeSentenceProbability = 1
    start_token_index = t.word_index[start_token]
    
    for number in range(0,max_decoder_seq_length,BEAM_LENGTH):
        if states_value is None:
            break
        partialSentence, states_value, currentCumulativeSentenceProbability = beamSearch(start_token_index, states_value, 1, min(BEAM_LENGTH, max_decoder_seq_length-number))
        if(len(partialSentence)>0):
            start_token_index = partialSentence[-1]
        cumulativeSentenceProbability *= currentCumulativeSentenceProbability
        sentenceIndices = sentenceIndices + partialSentence
        
    decodedSentence = ''
    for index in sentenceIndices:
        if index == 0:
            continue
        decodedSentence += t.index_word[index] + ' '
        
    return decodedSentence

    